In [2]:

# creating a virtual assistant 

#talk package
import pyttsx3 as ttx

# listen package
import speech_recognition as sr


#predictions packages
import json
import numpy as np
import pickle
import random
import nltk
from  nltk.stem import WordNetLemmatizer
import tensorflow as tf

In [3]:

class Assistant_SM:
    def __init__(self):
        #talk
        self.engine= ttx.init()
        self.voice= self.engine.getProperty("voices")
        self.engine.setProperty("voice", (self.voice)[0].id)
        
        #listen
        self.listener= sr.Recognizer()
        self.voice_noted=""
        
        #predictions attributes
        
        self.lemmatizer=WordNetLemmatizer()
        #the json intents file  
        # you can create a new one for you
        self.intents=json.loads(open("intents.json").read()) 
        # the lematized words of each  tag pattern  of json file 
        self.words= pickle.load(open("words.pkl", "rb"))
        self.classes=pickle.load(open("classes.pkl", "rb"))
        # the trained model
        self.model= tf.keras.models.load_model("chatBot.h5")
        
    #talk method
    def A_talk(self, text):
            self.engine.say(text)
            self.engine.runAndWait()
            
    
       
    # the listener method
    def A_listener(self):
    
        try:
            with sr.Microphone() as source:
                voix= (self.listener).listen(source)
                self.voice_noted=(self.listener).recognize_google(voix, language="fr-FR")
                self.voice_noted.lower() 
                
        except:
            pass 
        return self.voice_noted 
    
    # set of methods for predictions
    
    #1) mthod for lematization and tokenization
    def A_clean(self,sentence):
        sentence_words= nltk.word_tokenize(sentence)
        sentence_words= [(self.lemmatizer).lemmatize(word) for word in sentence_words]
        return sentence_words
    
    #2) set of words of the new sentence displayed 
    def A_bag(self, sentence):
        sentence_words= self.A_clean(sentence)
        bag= [0]*len(self.words)
        for w in sentence_words:
            for i , word in enumerate(self.words):
                if word==w:
                    bag[i]=1
        return np.array(bag)
    #3) method for probabilities predictions return the highest probabilities of a tag designed for a pattern
    def A_pred_p(self,sentence):
        bow= self.A_bag(sentence)
        res= (self.model).predict(np.array([bow]))[0]
        tsd=0.125
        result=[[i,r] for i,r in enumerate(res) if r>tsd]
        result.sort(key=lambda x:x[1], reverse=True)
        result_list=[]
        for r in result:
            result_list.append({"intent":self.classes[r[0]], "prob":str(r[1])})
        return result_list
    
    #4) the response method from chatbot 
    def A_getResponse(self ,intent_list, intents_json):
        result="je ne comprends pas , veuillez reformuler"
        tag= intent_list[0]["intent"]
        intents_list= intents_json["intents"]
        for i in intents_list:
            if i["tag"]==tag:
                result=random.choice(i["responses"])
                break
        return result 

In [4]:
#test the assistant
Assistant = Assistant_SM()
def launch_assistant(assistant):
    print("enregistrement")
    message=Assistant.A_listener()
    print("fin enregistrement")
    if assistant.voice_noted != "" :
        print(message)
        ints= Assistant.A_pred_p(message)
        res= Assistant.A_getResponse(ints, Assistant.intents)
        print(res)
        Assistant.A_talk(res)
        assistant.voice_noted = ""
        
  

In [ ]:

while True:
    launch_assistant(Assistant)


enregistrement
fin enregistrement
ah bon ok pour samedi la liste
1/1 [==============================] - 0s 278ms/step
 bonjour ravie de vous rencontrer 
enregistrement
fin enregistrement
enregistrement
